In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Set the parameters to search for a hotel in Paris.
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key,
    "location": "48.8566, 2.3522"}
# Use base URL to search for hotels in Paris.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# Make request and get the JSON data from the search.
hotels = requests.get(base_url, params=params).json()
hotels

{'html_attributions': [],
 'next_page_token': 'CqQCHAEAAHZiy0yviM0QIo6CvdyJaxBhsDVZgyODNLf_69dIRYFRWJuHeIRJlLo9DNEdeYWajSwJZu9krEg_Qq5KKqi1vXKa-aRsINsxj4VyfxZGDy7weU5T51dW9F8ay7R0Tv8GNE1FiMfgrW6yr0lQpaVb44irm30sLylVGxRhiAiG9OWTWNDhvxkKAnqZGAkTRetwtGcWtkv5W-PoE45Q_VuQM4RX1IZca_0ber69UTH6sbY5z_ar0PBD-IONiMuwPoa5oLc13djjX_s4H3J9C5po9SsHV7gZnmvDs6ByM9JyoKxO1ZiuUoOTfiFWPMd4WPvqvIQKGRpMmROczQzY-qtOQD_9LunQwPOErx_513qtd7rxYUE5ISRi0_PAolEyfWoK6xIQoAbCt8qYbloVhVtiXm33OhoUijrrj4HCsDV8auI-Z02he_JsAsY',
 'results': [{'business_status': 'OPERATIONAL',
   'geometry': {'location': {'lat': 48.8581126, 'lng': 2.3529277},
    'viewport': {'northeast': {'lat': 48.8594402302915,
      'lng': 2.354348530291502},
     'southwest': {'lat': 48.8567422697085, 'lng': 2.351650569708498}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png',
   'id': '7fa47cb61d7345cecc3b802d0e04fcff2b3d5ecb',
   'name': 'Hôtel Duo',
   'opening_hours': {'open_now': True},
   'photos': [{'height': 3840,

In [3]:
len(hotels["results"])

20

In [4]:
hotels["results"][0]["name"]

'Hôtel Duo'

In [5]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Busselton,AU,2020-04-24 03:09:33,-33.65,115.33,70.00,51,26,12.44
1,1,Hilo,US,2020-04-24 03:08:44,19.73,-155.09,80.60,57,40,9.17
2,2,Upernavik,GL,2020-04-24 03:09:33,72.79,-56.15,24.80,97,3,7.78
3,3,Albany,US,2020-04-24 03:05:46,42.60,-73.97,44.60,45,75,6.93
4,4,Rikitea,PF,2020-04-24 03:09:34,-23.12,-134.97,76.01,77,24,12.17


In [6]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= 90) & \
                                       (city_data_df["Max Temp"] >= 75)]
preferred_cities_df.head()


,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
1,1,Hilo,US,2020-04-24 03:08:44,19.73,-155.09,80.60,57,40,9.17
4,4,Rikitea,PF,2020-04-24 03:09:34,-23.12,-134.97,76.01,77,24,12.17
5,5,Saint-Philippe,RE,2020-04-24 03:09:34,-21.36,55.77,75.20,88,20,10.29
11,11,Amahai,ID,2020-04-24 03:09:36,-3.33,128.92,87.57,63,13,3.87
16,16,Kui Buri,TH,2020-04-24 03:09:37,12.08,99.85,87.80,62,20,5.82


In [7]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head

<bound method NDFrame.head of                City Country  Max Temp    Lat     Lng Hotel Name
1              Hilo      US     80.60  19.73 -155.09           
4           Rikitea      PF     76.01 -23.12 -134.97           
5    Saint-Philippe      RE     75.20 -21.36   55.77           
11           Amahai      ID     87.57  -3.33  128.92           
16         Kui Buri      TH     87.80  12.08   99.85           
..              ...     ...       ...    ...     ...        ...
557         Atambua      ID     83.57  -9.11  124.89           
562        Flinders      AU     82.00 -34.58  150.86           
565          Ilheus      BR     77.00 -14.79  -39.05           
568        Kalabagh      PK     84.09  32.97   71.56           
573         Bairiki      KI     87.80   1.33  172.98           

[155 rows x 6 columns]>

In [8]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
try:
    hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
except (IndexError):
    print("Hotel not found... skipping.")


In [9]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Add pop up markers
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [10]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [11]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))